In [ ]:
# !cp /kaggle/input/data-bowl-2019-external-data/*.py /kaggle/working

In [9]:
%reload_ext autoreload
%autoreload 2
import warnings
import jupytools.syspath
def ignore(*args, **kwargs): pass
warnings.warn = ignore
jupytools.syspath.add('..')

In [10]:
from collections import Counter, OrderedDict
from functools import partial

import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import GroupKFold
from tqdm.auto import tqdm

import bundle
import features as F
import selection
import utils as U
from dataset import load, load_sample, Subset
from encode import encode
from training import train, inference, submit, EnsembleTrainer, get_default_config
from meta import compute_meta_data
from metric import optimize_rounding_bounds, make_cappa_metric
from normalize import normalize

In [3]:
sample = False
if U.on_kaggle():
    U.log('Loading test set only.')
    tst_data = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
else:
    if sample:
        U.log('Warning: loading train and test data sample.')
        trn_data, _, _ = load_sample(Subset.Train, 500_000)
        [tst_data] = load_sample(Subset.Test, 500_000)
    else:
        U.log('Loading train and test.')
        trn_data, trn_spec, trn_targ = load(Subset.Train)
        [tst_data] = load(Subset.Test)

Loading train and test.
(11341042, 11) (17690, 7) (386, 3) (1156414, 11) 

In [4]:
transform = U.combine(
    partial(F.add_feature_combinations, pairs=[('title', 'event_code'),
                                               ('title', 'world'),
                                               ('title', 'type'),
                                               ('world', 'type')]),
    partial(F.add_datetime, column='timestamp', prefix='ts'),
)

if U.on_kaggle():
    U.log('Transforming test data only.')
    X_tst = transform(tst_data.copy())
    U.log(X_tst.shape)
else:
    U.log('Transforming train and test data.')
    X_tst = transform(tst_data.copy())
    X_trn = transform(trn_data.copy())
    U.log(X_trn.shape, X_tst.shape)

Transforming train and test data.
(11341042, 22) (1156414, 22)


In [5]:
if U.on_kaggle():
    U.log('Reading pre-computed meta from disk.')
    meta = bundle.meta()
else:
    U.log('Computing meta using train and test datasets.')
    meta = compute_meta_data(X_trn, X_tst)
    U.log('Saving computed meta on disk.')
    bundle.save_meta(meta, 'meta')

Computing meta using train and test datasets.
Saving computed meta on disk.


In [6]:
extractor = F.FeaturesExtractor([
    F.CountingFeatures(meta),
    F.PerformanceFeatures(meta),
    F.VarietyFeatures(meta),
    F.EventDataFeatures(meta),
    F.FeedbackFeatures(meta),
    F.TimestampFeatures2(meta),
    # F.ZFeatures(meta)
])

algo = F.InMemoryAlgorithm(extractor, meta, num_workers=12)

cat_cols = ['session_title','cnt_most_freq_title', 'cnt_least_freq_title', 
            'cnt_most_freq_world', 'cnt_least_freq_world', 'cnt_most_freq_type', 
            'cnt_least_freq_type', 'cnt_most_freq_title_world', 
            'cnt_least_freq_title_world', 'cnt_most_freq_title_type', 
            'cnt_least_freq_title_type', 'cnt_most_freq_world_type', 
            'cnt_least_freq_world_type']

if U.on_kaggle():
    U.log('Preparing test dataset.')
    X_tst = algo.run(X_tst, test=True)
    encoders = bundle.encoders()
    X_tst, _ = encode(X_tst, cat_cols, encoders=encoders)
else:
    U.log('Preparing train and test datasets.')
    X_trn = algo.run(X_trn)
    X_tst = algo.run(X_tst, test=True)
    X_trn, encoders = encode(X_trn, cat_cols)
    X_tst, _ = encode(X_tst, cat_cols, encoders=encoders)
    bundle.save(encoders, 'encoders')

Preparing train and test datasets.
Running algorithm in train mode.



Running algorithm in test mode.


In [9]:
if U.on_kaggle():
    U.log('Running post-processing on test set only.')
    F.add_user_wise_features(X_tst, meta)
else:
    U.log('Running post-processing on train and test sets.')
    F.add_user_wise_features(X_trn, meta)
    F.add_user_wise_features(X_tst, meta)

Running post-processing on train and test sets.


In [11]:
def encode_multi_target(dataset, feature, target='accuracy_group', smoothing=3.0):
    from category_encoders import TargetEncoder
    mappings = []
    for i in (1, 2, 3):
        target_col = dataset[target].map(lambda x: int(x == i))
        te = TargetEncoder(cols=[feature], smoothing=smoothing)
        te.fit(dataset, target_col)
        mapping = te.mapping[feature]
        mapping = {k-1: v for k, v in mapping.to_dict().items() if k > 0}
        dataset[f'te_{feature}_{i}'] = dataset[feature].map(mapping)
        mappings.append(mapping)
    return feature, mappings

if U.on_kaggle():
    te = bundle.target_encoders()
    for feature, mappings in te.items():
        for i, mapping in enumerate(mappings, 1):
            X_tst[f'te_{feature}_{i}'] = X_tst[feature].map(mapping)
else:
    target_encoders = {}
    feature, mappings = encode_multi_target(X_trn, feature='session_title')
    target_encoders[feature] = mappings
    for feature, mappings in target_encoders.items():
        for i, mapping in enumerate(mappings, 1):
            X_tst[f'te_{feature}_{i}'] = X_tst[feature].map(mapping)
    bundle.save(target_encoders, 'target_encoders')

In [ ]:
# if not U.on_kaggle():
#     with pd.option_context('display.max_rows', 2000):
#         display(X_trn.sample(3).T)

In [12]:
if not U.on_kaggle():
    X_trn.to_pickle('/tmp/X_trn.pickle')
    X_tst.to_pickle('/tmp/X_tst.pickle')

In [3]:
if not U.on_kaggle():
    X_trn = pd.read_pickle('/tmp/X_trn.pickle')
    X_tst = pd.read_pickle('/tmp/X_tst.pickle')

In [ ]:
# group_col = 'installation_id'
# U.log(f'Normalizing dataset using column for grouping: {group_col}')
# norm_dataset = X_tst if U.on_kaggle() else X_trn
# cnt_cols = U.starts_with(norm_dataset.columns, 'cnt_')
# normalize(norm_dataset, cnt_cols, grouping_key=group_col, method='min-max')

## Features Selection

In [4]:
selector = selection.FeatureSelection(
    rules=[
        ('nonzero', selection.non_zero_rows_and_cols),
    ],
    ignore_cols=[
        'accuracy_group', 
        'installation_id', 
        'game_session'
    ]
)
if U.on_kaggle():
    U.log('Loading relevant features list from disk.')
    features = bundle.features()
else:
    U.log('Deriving relevant features from train dataset.')
    features = selector.select(X_trn)
    bundle.save(features, 'features')

Deriving relevant features from train dataset.
Excluding from consideration: ['accuracy_group', 'installation_id', 'game_session']
Applying feature selection rule: nonzero
Selected features: 1109 of 1145
Keeping only features, selected by every rule.
Final number of features changed from 1145 to 1109


## Submission

In [ ]:
bundle.save({
    'rounders': rounders, 
    'ensemble': [result.models for result in ensemble]
}, key='lightgbm')

In [14]:
bundle.package(folder='/home/ck/data/bowl2019/external/')

Packaging training results into dataset.
/tmp/bowl2019/meta.joblib --> /home/ck/data/bowl2019/external/meta.joblib
/tmp/bowl2019/lightgbm.joblib --> /home/ck/data/bowl2019/external/lightgbm.joblib
/tmp/bowl2019/target_encoders.joblib --> /home/ck/data/bowl2019/external/target_encoders.joblib
/tmp/bowl2019/bounds.joblib --> /home/ck/data/bowl2019/external/bounds.joblib
/tmp/bowl2019/features.joblib --> /home/ck/data/bowl2019/external/features.joblib
/tmp/bowl2019/encoders.joblib --> /home/ck/data/bowl2019/external/encoders.joblib
/tmp/bowl2019/models_lightgbm_24.joblib --> /home/ck/data/bowl2019/external/models_lightgbm_24.joblib
/tmp/bowl2019/models_lightgbm_023.joblib --> /home/ck/data/bowl2019/external/models_lightgbm_023.joblib
Packaging helper scripts into dataset.
../selection.py --> /home/ck/data/bowl2019/external/selection.py
../encode.py --> /home/ck/data/bowl2019/external/encode.py
../features.py --> /home/ck/data/bowl2019/external/features.py
../training.py --> /home/ck/data/

'/home/ck/data/bowl2019/external/'

In [ ]:
if U.on_kaggle():
    U.log('Inference on Kaggle.')
    features = bundle.features()
    lightgbm = bundle.load('lightgbm')
    labels = []
    U.log('Running model predictions.')
    for rounder in lightgbm['rounders']:
        U.log(f'Using rounding method: {rounder.method}')
        for models in lightgbm['ensemble']:
            for model in models:
                model_pred = model.predict(X_tst[features])
                model_labels = rounder.predict(model_pred)
                labels.append(model_labels)
    labels = np.column_stack(labels)
    predictions = []
    U.log('Majority vote.')
    for label in labels:
        majority = Counter(label).most_common(1)[0][0]
        predictions.append(majority)
    U.log('Saving predictions on disk.')
    filename = submit(predictions)
    submit_df = pd.read_csv(filename)
    U.log('First 20 submission rows:')
    display(submit_df.head(20))
            
    #features = bundle.features()
    #bounds = bundle.bounds()
    #predicted = inference(X_tst, features, bounds=bounds, model=algo, version=version)
    #U.log('Saving predictions on disk.')
    #filename = submit(predicted)
    #submit_df = pd.read_csv(filename)
    #U.log('First 20 submission rows:')
    #display(submit_df.head(20))

In [ ]:
rounders = [Rounder(method='dist'), Rounder(method='optimal')]
labels = []
for rounder in rounders:
    for i, result in enumerate(ensemble, 1):
        rounder.fit(X_trn['accuracy_group'], result.oof)
        for j, model in enumerate(result.models, 1):
            U.log(f'Rounder: {rounder.method}, ensemble #{i}, model #{j}')
            model_pred = model.predict(X_tst[features])
            model_labels = rounder.predict(model_pred)
            labels.append(model_labels)

In [6]:
from rounding import Rounder

In [7]:
def seed_all(value):
    import random
    random.seed(value)
    np.random.seed(value)

In [4]:
features = bundle.features()

In [12]:
ensemble = []
U.log('Start training an ensemble of models with various random seeds.')
trainer = EnsembleTrainer(algo='lightgbm', eval_metric='rmse')
fold = GroupKFold(n_splits=5)
for i in range(5):
    seed = np.random.randint(0, 1001)
    U.log('=' * 80)
    U.log(f'Training model with random seed: {seed}')
    seed_all(seed)
    config = get_default_config('lightgbm')
    U.set_nested(config, 'model_params.seed', seed)
    U.set_nested(config, 'model_params.feature_fraction', 0.5)
    U.set_nested(config, 'model_params.bagging_fraction', 0.6)
    U.set_nested(config, 'model_params.bagging_freq', 1)
    U.set_nested(config, 'model_params.learning_rate', 0.001)
    U.set_nested(config, 'model_params.n_estimators', 10000)
    U.set_nested(config, 'fit_params.verbose', 250)
    U.set_nested(config, 'fit_params.early_stopping_rounds', 250)
    result = trainer.train(X_trn, features=features, fold=fold, config=config)
    ensemble.append(result)

Start training an ensemble of models with various random seeds.
Training model with random seed: 449
Running k-fold 1 of 5
Training until validation scores don't improve for 250 rounds.
[250]	trn's rmse: 1.19317	val's rmse: 1.20065
[500]	trn's rmse: 1.14496	val's rmse: 1.15504
[750]	trn's rmse: 1.10712	val's rmse: 1.11971
[1000]	trn's rmse: 1.07674	val's rmse: 1.09232
[1250]	trn's rmse: 1.05262	val's rmse: 1.07114
[1500]	trn's rmse: 1.03259	val's rmse: 1.05411
[1750]	trn's rmse: 1.01524	val's rmse: 1.03985
[2000]	trn's rmse: 1.00129	val's rmse: 1.0292
[2250]	trn's rmse: 0.989916	val's rmse: 1.02097
[2500]	trn's rmse: 0.979646	val's rmse: 1.01386
[2750]	trn's rmse: 0.970456	val's rmse: 1.0078
[3000]	trn's rmse: 0.962592	val's rmse: 1.003
[3250]	trn's rmse: 0.955533	val's rmse: 0.99911
[3500]	trn's rmse: 0.949349	val's rmse: 0.995999
[3750]	trn's rmse: 0.943778	val's rmse: 0.993475
[4000]	trn's rmse: 0.938579	val's rmse: 0.991407
[4250]	trn's rmse: 0.933805	val's rmse: 0.989643
[4500]	tr

[3250]	trn's rmse: 0.946104	val's rmse: 1.03706
[3500]	trn's rmse: 0.939825	val's rmse: 1.03465
[3750]	trn's rmse: 0.934281	val's rmse: 1.03285
[4000]	trn's rmse: 0.929073	val's rmse: 1.03129
[4250]	trn's rmse: 0.924308	val's rmse: 1.03011
[4500]	trn's rmse: 0.91977	val's rmse: 1.02905
[4750]	trn's rmse: 0.915552	val's rmse: 1.0282
[5000]	trn's rmse: 0.91157	val's rmse: 1.02745
[5250]	trn's rmse: 0.907746	val's rmse: 1.02686
[5500]	trn's rmse: 0.904176	val's rmse: 1.02634
[5750]	trn's rmse: 0.900717	val's rmse: 1.02592
[6000]	trn's rmse: 0.897324	val's rmse: 1.02552
[6250]	trn's rmse: 0.894076	val's rmse: 1.02524
[6500]	trn's rmse: 0.890899	val's rmse: 1.02497
[6750]	trn's rmse: 0.887868	val's rmse: 1.02473
[7000]	trn's rmse: 0.884945	val's rmse: 1.02445
[7250]	trn's rmse: 0.882067	val's rmse: 1.02433
[7500]	trn's rmse: 0.879261	val's rmse: 1.02412
[7750]	trn's rmse: 0.87654	val's rmse: 1.02388
[8000]	trn's rmse: 0.873865	val's rmse: 1.02378
[8250]	trn's rmse: 0.871246	val's rmse: 1.02

[4750]	trn's rmse: 0.918874	val's rmse: 1.0138
[5000]	trn's rmse: 0.914892	val's rmse: 1.01322
[5250]	trn's rmse: 0.911131	val's rmse: 1.01267
[5500]	trn's rmse: 0.907455	val's rmse: 1.01231
[5750]	trn's rmse: 0.903904	val's rmse: 1.01188
[6000]	trn's rmse: 0.900549	val's rmse: 1.01166
[6250]	trn's rmse: 0.897321	val's rmse: 1.01142
[6500]	trn's rmse: 0.894108	val's rmse: 1.01114
[6750]	trn's rmse: 0.891088	val's rmse: 1.01101
[7000]	trn's rmse: 0.888131	val's rmse: 1.01081
[7250]	trn's rmse: 0.88521	val's rmse: 1.01072
[7500]	trn's rmse: 0.882431	val's rmse: 1.01067
[7750]	trn's rmse: 0.879663	val's rmse: 1.01063
[8000]	trn's rmse: 0.87696	val's rmse: 1.0106
[8250]	trn's rmse: 0.874347	val's rmse: 1.01055
[8500]	trn's rmse: 0.871778	val's rmse: 1.01046
[8750]	trn's rmse: 0.869249	val's rmse: 1.01054
Early stopping, best iteration is:
[8573]	trn's rmse: 0.87103	val's rmse: 1.01045
Running k-fold 5 of 5
Training until validation scores don't improve for 250 rounds.
[250]	trn's rmse: 1.1

[6500]	trn's rmse: 0.901708	val's rmse: 0.981807
[6750]	trn's rmse: 0.898642	val's rmse: 0.981606
[7000]	trn's rmse: 0.895718	val's rmse: 0.981524
[7250]	trn's rmse: 0.892876	val's rmse: 0.981367
[7500]	trn's rmse: 0.890092	val's rmse: 0.981263
[7750]	trn's rmse: 0.88738	val's rmse: 0.981201
[8000]	trn's rmse: 0.884709	val's rmse: 0.981113
[8250]	trn's rmse: 0.882016	val's rmse: 0.981074
Early stopping, best iteration is:
[8208]	trn's rmse: 0.882459	val's rmse: 0.981055
Running k-fold 4 of 5
Training until validation scores don't improve for 250 rounds.
[250]	trn's rmse: 1.19154	val's rmse: 1.2067
[500]	trn's rmse: 1.14292	val's rmse: 1.16409
[750]	trn's rmse: 1.10213	val's rmse: 1.12897
[1000]	trn's rmse: 1.07118	val's rmse: 1.1033
[1250]	trn's rmse: 1.04618	val's rmse: 1.0834
[1500]	trn's rmse: 1.02549	val's rmse: 1.06755
[1750]	trn's rmse: 1.00834	val's rmse: 1.0552
[2000]	trn's rmse: 0.994416	val's rmse: 1.04599
[2250]	trn's rmse: 0.98229	val's rmse: 1.03846
[2500]	trn's rmse: 0.97

[500]	trn's rmse: 1.14718	val's rmse: 1.13556
[750]	trn's rmse: 1.10676	val's rmse: 1.10103
[1000]	trn's rmse: 1.07536	val's rmse: 1.07499
[1250]	trn's rmse: 1.0507	val's rmse: 1.0552
[1500]	trn's rmse: 1.03068	val's rmse: 1.03986
[1750]	trn's rmse: 1.01402	val's rmse: 1.0277
[2000]	trn's rmse: 1.00032	val's rmse: 1.01838
[2250]	trn's rmse: 0.988452	val's rmse: 1.01075
[2500]	trn's rmse: 0.978569	val's rmse: 1.00488
[2750]	trn's rmse: 0.969991	val's rmse: 1.00027
[3000]	trn's rmse: 0.962423	val's rmse: 0.99656
[3250]	trn's rmse: 0.955624	val's rmse: 0.993533
[3500]	trn's rmse: 0.949692	val's rmse: 0.991344
[3750]	trn's rmse: 0.944284	val's rmse: 0.989468
[4000]	trn's rmse: 0.939259	val's rmse: 0.988045
[4250]	trn's rmse: 0.934485	val's rmse: 0.986669
[4500]	trn's rmse: 0.930067	val's rmse: 0.985587
[4750]	trn's rmse: 0.925913	val's rmse: 0.98466
[5000]	trn's rmse: 0.921943	val's rmse: 0.983962
[5250]	trn's rmse: 0.918147	val's rmse: 0.983385
[5500]	trn's rmse: 0.91455	val's rmse: 0.982

[3750]	trn's rmse: 0.943105	val's rmse: 1.01053
[4000]	trn's rmse: 0.938005	val's rmse: 1.00858
[4250]	trn's rmse: 0.93313	val's rmse: 1.00677
[4500]	trn's rmse: 0.928736	val's rmse: 1.00549
[4750]	trn's rmse: 0.92454	val's rmse: 1.00433
[5000]	trn's rmse: 0.920572	val's rmse: 1.00334
[5250]	trn's rmse: 0.916799	val's rmse: 1.00255
[5500]	trn's rmse: 0.913196	val's rmse: 1.00184
[5750]	trn's rmse: 0.909765	val's rmse: 1.00133
[6000]	trn's rmse: 0.906447	val's rmse: 1.00094
[6250]	trn's rmse: 0.903252	val's rmse: 1.00054
[6500]	trn's rmse: 0.900151	val's rmse: 1.00025
[6750]	trn's rmse: 0.897172	val's rmse: 1
[7000]	trn's rmse: 0.894261	val's rmse: 0.999745
[7250]	trn's rmse: 0.891371	val's rmse: 0.999563
[7500]	trn's rmse: 0.888565	val's rmse: 0.999347
[7750]	trn's rmse: 0.885845	val's rmse: 0.999321
[8000]	trn's rmse: 0.883203	val's rmse: 0.99924
[8250]	trn's rmse: 0.880553	val's rmse: 0.999118
[8500]	trn's rmse: 0.878022	val's rmse: 0.999046
[8750]	trn's rmse: 0.875508	val's rmse: 0.

In [13]:
rounders = [Rounder(method='dist'), Rounder(method='optimal')]
labels = []
for rounder in rounders:
    for i, result in enumerate(ensemble, 1):
        rounder.fit(X_trn['accuracy_group'], result.oof)
        for j, model in enumerate(result.models, 1):
            U.log(f'Rounder: {rounder.method}, ensemble #{i}, model #{j}')
            model_pred = model.predict(X_tst[features])
            model_labels = rounder.predict(model_pred)
            labels.append(model_labels)

Rounder: dist, ensemble #1, model #1
Rounder: dist, ensemble #1, model #2
Rounder: dist, ensemble #1, model #3
Rounder: dist, ensemble #1, model #4
Rounder: dist, ensemble #1, model #5
Rounder: dist, ensemble #2, model #1
Rounder: dist, ensemble #2, model #2
Rounder: dist, ensemble #2, model #3
Rounder: dist, ensemble #2, model #4
Rounder: dist, ensemble #2, model #5
Rounder: dist, ensemble #3, model #1
Rounder: dist, ensemble #3, model #2
Rounder: dist, ensemble #3, model #3
Rounder: dist, ensemble #3, model #4
Rounder: dist, ensemble #3, model #5
Rounder: dist, ensemble #4, model #1
Rounder: dist, ensemble #4, model #2
Rounder: dist, ensemble #4, model #3
Rounder: dist, ensemble #4, model #4
Rounder: dist, ensemble #4, model #5
Rounder: dist, ensemble #5, model #1
Rounder: dist, ensemble #5, model #2
Rounder: dist, ensemble #5, model #3
Rounder: dist, ensemble #5, model #4
Rounder: dist, ensemble #5, model #5
Rounder: optimal, ensemble #1, model #1
Rounder: optimal, ensemble #1, mode

In [15]:
bundle.save({
    'rounders': rounders, 
    'ensemble': [result.models for result in ensemble]
}, key='lightgbm')

'/tmp/bowl2019/lightgbm.joblib'

In [11]:
from multiprocessing import Queue, Process

In [ ]:
ensemble = []
U.log('Start training an ensemble of models with various random seeds.')
trainer = EnsembleTrainer(algo='xgboost', eval_metric='rmse')
fold = GroupKFold(n_splits=5)
for i in range(5):
    seed = np.random.randint(0, 1001)
    U.log('=' * 80)
    U.log(f'Training model with random seed: {seed}')
    seed_all(seed)
    config = get_default_config('xgboost')
    U.set_nested(config, 'model_params.seed', seed)
    U.set_nested(config, 'model_params.max_depth', 6)
    U.set_nested(config, 'model_params.subsample', 0.5)
    U.set_nested(config, 'model_params.colsample_bytree', 0.8)
    U.set_nested(config, 'model_params.colsample_bylevel', 0.6)
    U.set_nested(config, 'model_params.colsample_bynode', 0.5)
    U.set_nested(config, 'model_params.learning_rate', 0.003)
    U.set_nested(config, 'model_params.n_estimators', 5000)
    U.set_nested(config, 'model_params.lambda', 1)
    U.set_nested(config, 'model_params.alpha', 1)
    U.set_nested(config, 'model_params.objective', 'reg:squarederror')
    # U.set_nested(config, 'model_params.gpu_id', 1)
    U.set_nested(config, 'model_params.nthread', 12)
    U.set_nested(config, 'model_params.tree_method', 'hist')
    U.set_nested(config, 'fit_params.verbose', 100)
    U.set_nested(config, 'fit_params.early_stopping_rounds', 100)
    result = trainer.train(X_trn, features=features, fold=fold, config=config)
    ensemble.append(result)

Start training an ensemble of models with various random seeds.
Training model with random seed: 875
Running k-fold 1 of 5
[0]	validation_0-rmse:1.86716	validation_1-rmse:1.86478
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:1.53987	validation_1-rmse:1.5447
[200]	validation_0-rmse:1.3199	validation_1-rmse:1.33309
[300]	validation_0-rmse:1.17516	validation_1-rmse:1.19777
[400]	validation_0-rmse:1.08176	validation_1-rmse:1.1134
[500]	validation_0-rmse:1.0218	validation_1-rmse:1.0622
[600]	validation_0-rmse:0.982178	validation_1-rmse:1.03073
[700]	validation_0-rmse:0.954875	validation_1-rmse:1.01114
[800]	validation_0-rmse:0.935723	validation_1-rmse:0.999103
[900]	validation_0-rmse:0.921191	validation_1-rmse:0.991458
[1000]	validation_0-rmse:0.909429	validation_1-rmse:0.986045
[1100]	validation_0-rmse:0.899873	validation_1-rmse:0.982395
[1200]	validatio

[700]	validation_0-rmse:0.946975	validation_1-rmse:1.04265
[800]	validation_0-rmse:0.92722	validation_1-rmse:1.03273
[900]	validation_0-rmse:0.912319	validation_1-rmse:1.02636
[1000]	validation_0-rmse:0.900447	validation_1-rmse:1.0224
[1100]	validation_0-rmse:0.890746	validation_1-rmse:1.01983
[1200]	validation_0-rmse:0.882211	validation_1-rmse:1.01781
[1300]	validation_0-rmse:0.874735	validation_1-rmse:1.01604
[1400]	validation_0-rmse:0.867582	validation_1-rmse:1.0149
[1500]	validation_0-rmse:0.861279	validation_1-rmse:1.01391
[1600]	validation_0-rmse:0.855259	validation_1-rmse:1.01324
[1700]	validation_0-rmse:0.849783	validation_1-rmse:1.01267
[1800]	validation_0-rmse:0.844541	validation_1-rmse:1.01224
[1900]	validation_0-rmse:0.83932	validation_1-rmse:1.01183
[2000]	validation_0-rmse:0.834461	validation_1-rmse:1.01159
[2100]	validation_0-rmse:0.829706	validation_1-rmse:1.01123
[2200]	validation_0-rmse:0.825177	validation_1-rmse:1.01112
[2300]	validation_0-rmse:0.82066	validation_1-r

Stopping. Best iteration:
[2300]	validation_0-rmse:0.826902	validation_1-rmse:0.993519

Running k-fold 5 of 5
[0]	validation_0-rmse:1.8688	validation_1-rmse:1.85834
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:1.5387	validation_1-rmse:1.5453
[200]	validation_0-rmse:1.31682	validation_1-rmse:1.34134
[300]	validation_0-rmse:1.17066	validation_1-rmse:1.21252
[400]	validation_0-rmse:1.07598	validation_1-rmse:1.13372
[500]	validation_0-rmse:1.01446	validation_1-rmse:1.08652
[600]	validation_0-rmse:0.974272	validation_1-rmse:1.05906
[700]	validation_0-rmse:0.946742	validation_1-rmse:1.04271
[800]	validation_0-rmse:0.927338	validation_1-rmse:1.03273
[900]	validation_0-rmse:0.912528	validation_1-rmse:1.02677
[1000]	validation_0-rmse:0.900676	validation_1-rmse:1.02289
[1100]	validation_0-rmse:0.890753	validation_1-rmse:1.02023
[1200]	validation_0-rmse:0.8819

[1100]	validation_0-rmse:0.89515	validation_1-rmse:1.00328
[1200]	validation_0-rmse:0.886525	validation_1-rmse:1.00076
[1300]	validation_0-rmse:0.879211	validation_1-rmse:0.999228
[1400]	validation_0-rmse:0.872431	validation_1-rmse:0.997996
[1500]	validation_0-rmse:0.866284	validation_1-rmse:0.996891
[1600]	validation_0-rmse:0.860377	validation_1-rmse:0.996053
[1700]	validation_0-rmse:0.854955	validation_1-rmse:0.995419
[1800]	validation_0-rmse:0.849975	validation_1-rmse:0.994796
[1900]	validation_0-rmse:0.844831	validation_1-rmse:0.994379
[2000]	validation_0-rmse:0.840072	validation_1-rmse:0.994109
[2100]	validation_0-rmse:0.835304	validation_1-rmse:0.993533
[2200]	validation_0-rmse:0.83098	validation_1-rmse:0.99316
[2300]	validation_0-rmse:0.826652	validation_1-rmse:0.992864
[2400]	validation_0-rmse:0.82232	validation_1-rmse:0.992755
[2500]	validation_0-rmse:0.818013	validation_1-rmse:0.992466
[2600]	validation_0-rmse:0.813778	validation_1-rmse:0.992379
[2700]	validation_0-rmse:0.809

[1100]	validation_0-rmse:0.901355	validation_1-rmse:0.986747
[1200]	validation_0-rmse:0.893271	validation_1-rmse:0.984449
[1300]	validation_0-rmse:0.886119	validation_1-rmse:0.98285
[1400]	validation_0-rmse:0.879181	validation_1-rmse:0.98145
[1500]	validation_0-rmse:0.873083	validation_1-rmse:0.980405
[1600]	validation_0-rmse:0.867079	validation_1-rmse:0.979536
[1700]	validation_0-rmse:0.861658	validation_1-rmse:0.979025
[1800]	validation_0-rmse:0.8564	validation_1-rmse:0.978632
[1900]	validation_0-rmse:0.851767	validation_1-rmse:0.978208
[2000]	validation_0-rmse:0.846917	validation_1-rmse:0.977889
[2100]	validation_0-rmse:0.842615	validation_1-rmse:0.977678
[2200]	validation_0-rmse:0.838117	validation_1-rmse:0.977463
[2300]	validation_0-rmse:0.833829	validation_1-rmse:0.977541
Stopping. Best iteration:
[2214]	validation_0-rmse:0.837582	validation_1-rmse:0.977401

Running k-fold 4 of 5
[0]	validation_0-rmse:1.87449	validation_1-rmse:1.83516
Multiple eval metrics have been passed: 'vali

In [ ]:
rounders = [Rounder(method='dist'), Rounder(method='optimal')]
labels = []
for rounder in rounders:
    for i, result in enumerate(ensemble, 1):
        rounder.fit(X_trn['accuracy_group'], result.oof)
        for j, model in enumerate(result.models, 1):
            U.log(f'Rounder: {rounder.method}, ensemble #{i}, model #{j}')
            model_pred = model.predict(X_tst[features])
            model_labels = rounder.predict(model_pred)
            labels.append(model_labels)

In [ ]:
bundle.save({
    'rounders': rounders, 
    'ensemble': [result.models for result in ensemble]
}, key='xgboost')

In [ ]:
algo = 'lightgbm'
version = '024'

if U.on_kaggle():
    U.log('Inference on Kaggle.')
    features = bundle.features()
    bounds = bundle.bounds()
    predicted = inference(X_tst, features, bounds=bounds, model=algo, version=version)
    U.log('Saving predictions on disk.')
    filename = submit(predicted)
    submit_df = pd.read_csv(filename)
    U.log('First 20 submission rows:')
    display(submit_df.head(20))
    
else:
    U.log(f'Training model: {algo}')
    cappa = make_cappa_metric(X_trn['accuracy_group'])
    trainer = EnsembleTrainer(algo=algo, cv_metrics={'cappa': cappa})
    fold = GroupKFold(n_splits=5)
    config = get_default_config(algo)
    U.set_nested(config, 'model_params.feature_fraction', 0.4)
    U.set_nested(config, 'model_params.bagging_fraction', 0.5)
    U.set_nested(config, 'model_params.bagging_freq', 1)
    U.set_nested(config, 'model_params.learning_rate', 0.003)
    result = trainer.train(X_trn, features=features, fold=fold, config=config)
    U.log('Saving the trained models')
    bundle.save(result.models, f'models_{algo}_{version}')
    U.log('Saving the optimal rounding bounds')
    bounds = optimize_rounding_bounds(result.oof, X_trn['accuracy_group'].values)
    U.log(f'Optimal bounds: {bounds}')
    bundle.save(bounds, 'bounds')
    U.log(f'Final features number: {len(features)}')
    bundle.save(features, 'features')

In [ ]:
if not U.on_kaggle():
    import os
    features = bundle.features()
    bounds = bundle.bounds()
    filename = submit(inference(X_tst, features, bounds, model=algo, version=version))
    assert os.path.exists(filename)
    assert pd.read_csv(filename).shape[0] == 1000
    bundle.package(folder='/home/ck/data/bowl2019/external/')

In [ ]:
%matplotlib inline

In [ ]:
pd.read_csv('submission.csv')['accuracy_group'].value_counts().reset_index().sort_values(by='index').set_index('index').plot.bar()

In [ ]:
X_trn['accuracy_group'].value_counts().reset_index().sort_values(by='index').set_index('index').plot.bar()

## Stack Many Models

In [16]:
bundle.package(folder='/home/ck/data/bowl2019/external/')

Packaging training results into dataset.
/tmp/bowl2019/meta.joblib --> /home/ck/data/bowl2019/external/meta.joblib
/tmp/bowl2019/lightgbm.joblib --> /home/ck/data/bowl2019/external/lightgbm.joblib
/tmp/bowl2019/target_encoders.joblib --> /home/ck/data/bowl2019/external/target_encoders.joblib
/tmp/bowl2019/bounds.joblib --> /home/ck/data/bowl2019/external/bounds.joblib
/tmp/bowl2019/xgboost.joblib --> /home/ck/data/bowl2019/external/xgboost.joblib
/tmp/bowl2019/features.joblib --> /home/ck/data/bowl2019/external/features.joblib
/tmp/bowl2019/encoders.joblib --> /home/ck/data/bowl2019/external/encoders.joblib
/tmp/bowl2019/models_lightgbm_24.joblib --> /home/ck/data/bowl2019/external/models_lightgbm_24.joblib
/tmp/bowl2019/models_lightgbm_023.joblib --> /home/ck/data/bowl2019/external/models_lightgbm_023.joblib
Packaging helper scripts into dataset.
../selection.py --> /home/ck/data/bowl2019/external/selection.py
../encode.py --> /home/ck/data/bowl2019/external/encode.py
../features.py -

'/home/ck/data/bowl2019/external/'